In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

## 1. Column Group by
* 아래 세 단계를 적용하여 데이터 그룹화 수행 (SQL Group by 맥락)
    * 데이터 분할
    * operation 적용
    * 데이터 병합

In [5]:
df = pd.read_csv('./train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### 1. Groupby 수행
* 각 그룹과 그룹에 속한 index를 dict 형태로 반환

In [7]:
class_group = df.groupby('Pclass')

In [8]:
# group by한 데이터에 대하여 group 정보 출력
class_group.groups

{1: Int64Index([  1,   3,   6,  11,  23,  27,  30,  31,  34,  35,
             ...
             853, 856, 857, 862, 867, 871, 872, 879, 887, 889],
            dtype='int64', length=216),
 2: Int64Index([  9,  15,  17,  20,  21,  33,  41,  43,  53,  56,
             ...
             848, 854, 861, 864, 865, 866, 874, 880, 883, 886],
            dtype='int64', length=184),
 3: Int64Index([  0,   2,   4,   5,   7,   8,  10,  12,  13,  14,
             ...
             875, 876, 877, 878, 881, 882, 884, 885, 888, 890],
            dtype='int64', length=491)}

In [9]:
gender_group = df.groupby('Sex')
gender_group.groups

{'female': Int64Index([  1,   2,   3,   8,   9,  10,  11,  14,  15,  18,
             ...
             866, 871, 874, 875, 879, 880, 882, 885, 887, 888],
            dtype='int64', length=314),
 'male': Int64Index([  0,   4,   5,   6,   7,  12,  13,  16,  17,  20,
             ...
             873, 876, 877, 878, 881, 883, 884, 886, 889, 890],
            dtype='int64', length=577)}

#### 2. Groupping 함수
* 그룹 데이터에 적용 가능한 통계 함수 (단, NaN은 제외하여 연산)
    * count, sum, mean, std, var, min, max ... 등

In [10]:
class_group.count()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,,
1,216,216,216,216,186,216,216,216,216,176,214
2,184,184,184,184,173,184,184,184,184,16,184
3,491,491,491,491,355,491,491,491,491,12,491


In [11]:
class_group.sum()

,PassengerId,Survived,Age,SibSp,Parch,Fare
Pclass,,,,,,
1,99705,136,7111.42,90,77,18177.4125
2,82056,87,5168.83,74,70,3801.8417
3,215625,119,8924.92,302,193,6714.6951


In [14]:
# 객실 등급에 따른 생존율
class_group.mean()['Survived']

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

In [16]:
# df.groupby('Sex').mean()['Survived']
gender_group.mean()['Survived']

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

#### 3. 복수 컬럼 Groupping
* group by에 컬럼 리스트를 전달
* 반환 결과는 Multi-index를 가지는 Data frame

In [17]:
df.groupby(['Pclass', 'Sex']).mean()

PassengerId  Survived        Age     SibSp     Parch  \
Pclass Sex                                                            
1      female   469.212766  0.968085  34.611765  0.553191  0.457447   
       male     455.729508  0.368852  41.281386  0.311475  0.278689   
2      female   443.105263  0.921053  28.722973  0.486842  0.605263   
       male     447.962963  0.157407  30.740707  0.342593  0.222222   
3      female   399.729167  0.500000  21.750000  0.895833  0.798611   
       male     455.515850  0.135447  26.507589  0.498559  0.224784   

                     Fare  
Pclass Sex                 
1      female  106.125798  
       male     67.226127  
2      female   21.970121  
       male     19.741782  
3      female   16.118810  
       male     12.661633

In [18]:
# 2등급의 여성 데이터만 가져오기
df.groupby(['Pclass', 'Sex']).mean().loc[(2, 'female')]

PassengerId    443.105263
Survived         0.921053
Age             28.722973
SibSp            0.486842
Parch            0.605263
Fare            21.970121
Name: (2, female), dtype: float64

In [19]:
df.groupby(['Pclass', 'Sex']).mean()['Survived']

Pclass  Sex   
1       female    0.968085
        male      0.368852
2       female    0.921053
        male      0.157407
3       female    0.500000
        male      0.135447
Name: Survived, dtype: float64

## 2. Index Group by

#### 1. Index Grouppong
* **set_index** 함수
    * column 데이터를 Index 레벨로 변경
* **reset_index** 함수
    * 인덱스 초기화

In [23]:
# 명시된 컬럼을 대상으로 Index Level로 변환 수행.
df.set_index(['Pclass', 'Sex'])
#df.reset_index()

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [24]:
# index로 Group by 할 경우, 왼쪽부터 index level이 0부터 시작
# 즉, 만약 Age로 Index를 하고 이 Index로 Group by 수행 시 다음과 같음
df.set_index('Age').groupby(level=0).mean()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare
Age,,,,,,
0.42,804.000000,1.000000,3.000000,0.000000,1.000000,8.516700
0.67,756.000000,1.000000,2.000000,1.000000,1.000000,14.500000
0.75,557.500000,1.000000,3.000000,2.000000,1.000000,19.258300
0.83,455.500000,1.000000,2.000000,0.500000,1.500000,23.875000
0.92,306.000000,1.000000,1.000000,1.000000,2.000000,151.550000
1.00,415.428571,0.714286,2.714286,1.857143,1.571429,30.005957
2.00,346.900000,0.300000,2.600000,2.100000,1.300000,37.536250
3.00,272.000000,0.833333,2.500000,1.833333,1.333333,25.781950
4.00,466.100000,0.700000,2.600000,1.600000,1.400000,29.543330


In [26]:
#############################################
# 나이대별로 성별 구하기
############################################# 

import math
def transfer_age(age):
    if math.isnan(age):
        return -1
    return math.floor(age / 10) * 10

In [29]:
# Index를 사용할 경우, 다음과 같이 함수를 사용하여 group by할 수 있다.
df.set_index('Age').groupby(transfer_age).mean()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare
-1,435.581921,0.293785,2.598870,0.564972,0.180791,22.158567
0,424.741935,0.612903,2.629032,1.854839,1.403226,30.576679
10,444.362745,0.401961,2.470588,0.666667,0.470588,32.535132
20,433.231818,0.350000,2.450000,0.322727,0.250000,27.278937
30,472.449102,0.437126,2.113772,0.353293,0.329341,40.377294
40,465.606742,0.382022,1.966292,0.370787,0.471910,38.002297
50,440.187500,0.416667,1.562500,0.291667,0.270833,47.933333
60,433.736842,0.315789,1.473684,0.263158,0.368421,48.367542
70,496.500000,0.000000,1.833333,0.166667,0.166667,30.197233
80,631.000000,1.000000,1.000000,0.000000,0.000000,30.000000


#### 2. Multi-Index를 이용한 Groupping

In [33]:
df.set_index(['Pclass', 'Sex']).groupby(level=[0, 1]).mean()

PassengerId  Survived        Age     SibSp     Parch  \
Pclass Sex                                                            
1      female   469.212766  0.968085  34.611765  0.553191  0.457447   
       male     455.729508  0.368852  41.281386  0.311475  0.278689   
2      female   443.105263  0.921053  28.722973  0.486842  0.605263   
       male     447.962963  0.157407  30.740707  0.342593  0.222222   
3      female   399.729167  0.500000  21.750000  0.895833  0.798611   
       male     455.515850  0.135447  26.507589  0.498559  0.224784   

                     Fare  
Pclass Sex                 
1      female  106.125798  
       male     67.226127  
2      female   21.970121  
       male     19.741782  
3      female   16.118810  
       male     12.661633

#### 3. Aggregate (집계) 함수
*  group by 결과에 다양한 집계 함수를 사용하여 그룹별로 데이터 확인 가능

In [34]:
df.set_index(['Pclass', 'Sex']).groupby(level=[0, 1]).aggregate([np.mean, np.sum, np.max])

PassengerId               Survived                 Age           \
                     mean     sum amax      mean sum amax       mean      sum   
Pclass Sex                                                                      
1      female  469.212766   44106  888  0.968085  91    1  34.611765  2942.00   
       male    455.729508   55599  890  0.368852  45    1  41.281386  4169.42   
2      female  443.105263   33676  881  0.921053  70    1  28.722973  2125.50   
       male    447.962963   48380  887  0.157407  17    1  30.740707  3043.33   
3      female  399.729167   57561  889  0.500000  72    1  21.750000  2218.50   
       male    455.515850  158064  891  0.135447  47    1  26.507589  6706.42   

                        SibSp               Parch                  Fare  \
               amax      mean  sum amax      mean  sum amax        mean   
Pclass Sex                                                                
1      female  63.0  0.553191   52    3  0.457447   43    2  106.125798   
       male    80.0  0.311475   38    3  0.278689   34    4   67.226127   
2      female  57.0  0.486842   37    3  0.605263   46    3   21.970121   
       male    70.0  0.342593   37    2  0.222222   24    2   19.741782   
3      female  63.0  0.895833  129    8  0.798611  115    6   16.118810   
       male    74.0  0.498559  173    8  0.224784   78    5   12.661633   

                                    
                     sum      amax  
Pclass Sex                          
1      female  9975.8250  512.3292  
       male    8201.5875  512.3292  
2      female  1669.7292   65.0000  
       male    2132.1125   73.5000  
3      female  2321.1086   69.5500  
       male    4393.5865   69.5500

## 3. Transform 함수

* Group by 후 transform 함수 사용 시, 원래 Index를 유지한 상태로 통계 함수 적용
* 전체 데이터의 집계가 아닌 각 그룹에서의 집계 계산
    * 따라서 새로 생성된 데이터를 원본 Dataframe과 Merge하기 용이

In [38]:
df['Age2'] = df.groupby('Pclass').transform(np.mean)['Age']
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age2
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,25.140620
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,38.233441
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,25.140620
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,38.233441
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,25.140620
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,25.140620
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,38.233441
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,25.140620
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,25.140620
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,29.877630


In [42]:
df.groupby(['Pclass', 'Sex']).transform(np.mean)

,PassengerId,Survived,Age,SibSp,Parch,Fare,Age2
0,455.515850,0.135447,26.507589,0.498559,0.224784,12.661633,25.140620
1,469.212766,0.968085,34.611765,0.553191,0.457447,106.125798,38.233441
2,399.729167,0.500000,21.750000,0.895833,0.798611,16.118810,25.140620
3,469.212766,0.968085,34.611765,0.553191,0.457447,106.125798,38.233441
4,455.515850,0.135447,26.507589,0.498559,0.224784,12.661633,25.140620
5,455.515850,0.135447,26.507589,0.498559,0.224784,12.661633,25.140620
6,455.729508,0.368852,41.281386,0.311475,0.278689,67.226127,38.233441
7,455.515850,0.135447,26.507589,0.498559,0.224784,12.661633,25.140620
8,399.729167,0.500000,21.750000,0.895833,0.798611,16.118810,25.140620
9,443.105263,0.921053,28.722973,0.486842,0.605263,21.970121,29.877630
